In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("transformer.pdf")
data = loader.load()  

In [2]:
data

[Document(metadata={'source': 'transformer.pdf', 'page': 0}, page_content="Transformer   \nelectrical  transformer  is a static  electrical  machine  which transforms electrical power from one circuit to \nanother circuit, without changing the frequency. Transformer can increase or decrease the voltage with \ncorresponding decrease or increase in current.  \nWorking principle of transformer  \n \n The basic  principle  behind  working  of a transformer  is the phenomenon of mutual induction between two \nwindings linked by common magnetic flux. The figure at right shows the simplest form of a transformer. \nBasically a transformer consists of two inductive coils; primary winding and secondary winding. The coils are \nelectrically separated but magnetically linked to ea ch other. When, primary winding is connected to a source of \nalternating voltage, alternating  magnetic  flux is produced  around  the winding . The core provides magnetic  path \nfor the flux, to get linked with the se

In [3]:
len(data)

17

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  37


In [5]:
docs[0]

Document(metadata={'source': 'transformer.pdf', 'page': 0}, page_content='Transformer   \nelectrical  transformer  is a static  electrical  machine  which transforms electrical power from one circuit to \nanother circuit, without changing the frequency. Transformer can increase or decrease the voltage with \ncorresponding decrease or increase in current.  \nWorking principle of transformer  \n \n The basic  principle  behind  working  of a transformer  is the phenomenon of mutual induction between two \nwindings linked by common magnetic flux. The figure at right shows the simplest form of a transformer. \nBasically a transformer consists of two inductive coils; primary winding and secondary winding. The coils are \nelectrically separated but magnetically linked to ea ch other. When, primary winding is connected to a source of \nalternating voltage, alternating  magnetic  flux is produced  around  the winding . The core provides magnetic  path \nfor the flux, to get linked with the sec

# Get an API key: 

 https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

 Embedding models: https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/

In [ ]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings # Embedding Model

from dotenv import load_dotenv
load_dotenv() 

# Chunking

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
# 5 vector

c:\Users\HP\anaconda3\envs\ten\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [7]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [11]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What are the types of transformers")


In [12]:
len(retrieved_docs)

10

In [13]:
retrieved_docs

[Document(metadata={'page': 2, 'source': 'transformer.pdf'}, page_content='(ii) Shell type transformer  \nThe coils are former wound and mounted in layers stacked with insulation between them. A shell type \ntransformer may have simple rectangu lar form (as shown in above fig), or   it may have a distributed form.  \n \n(B) On the basis of their purpose  \n1. Step up transformer: Voltage increases (with subsequent decrease in current) at secondary.  \n2. Step down transformer: Voltage decreases (with subsequent increase in current) at secondary.  \n \n(C) On the basis of type of supply  \n1. Single phase transformer  \n2. Three phase transformer  \n3.  \n(D) On the basis of their use  \n1. Power transformer: Used in  transmission  network , high rating  \n2. Distribution transformer: Used in  distribution  network , comparatively lower rating than that of power \ntransformers.  \n3. Instrument transformer: Used in relay and protection purpose in different instrume nts in industries  \n

In [16]:
print(retrieved_docs[1].page_content)

at joints, laminatio ns are stacked by alternating the sides of joint. That is, if joints of first sheet assembly are at 
front face, the joints of following assemble are kept at back face.  
 
 
Types of transformers  
Transformers can be classified on different basis, like types of construction, types of cooling etc.  
 
(A) On the basis  of construction , transformers can be classified into two types as;  
(i) Core type transformer and (ii) Shell type transformer, which are described below.  
 
(i) Core type transformer  
In core typ e transformer, windings are cylindrical former wound, mounted on   the core limbs as shown in the 
figure above. The cylindrical coils have different layers and each layer is insulated from each other. Materials 
like paper, cloth or mica can be used for insu lation. Low voltage windings are placed nearer to the core, as they 
are easier to insulate.


In [25]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-002",temperature=0.3, max_tokens=500)

In [29]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer in bullet format"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [30]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [31]:
response = rag_chain.invoke({"input": "What are the types of transformers"})
print(response["answer"])

* Transformers can be classified based on construction (core type and shell type).
* They can also be classified based on their purpose (step-up and step-down).
* Other classifications include the type of supply (single-phase and three-phase) and their use (power, distribution, and instrument transformers).

